<a href="https://colab.research.google.com/github/jzanazzi249/Disk-Precession-Workbook/blob/main/Nodal_Precession_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import odeint
import matplotlib.pyplot as plt

#Nodal Precession Workbook

In this short demo, we will illustrate nodal precession, when an orbit of a test particle continuously changes direction due to the vertical force from an inclined, massive companion:

Prec_setup_v2.svg

# Circular Orbit

We first examine the test particle's circular orbit, ignoring the vertical force.  The equations of motion for the test particle are
\begin{equation}
\frac{d^2 𝙧}{d t^2} = - \frac{G M}{r^3} 𝙧 ,
\end{equation}
where $𝙧$ is the position, $t$ is the time, $G$ Newton's gravitational constant, $M$ the mass of the central object.  Numerically integrating the particle's trajectory, we can visualize the particle's path, starting with a position $r=R$ and velocity $v_0 = \sqrt{G M/R}$, letting the particle's inclination be $45^\circ$ from the $x$-$y$ plane. Color from blue to white displays time from $t=0$ to $t=t_{\rm end}$.

In [3]:
# Fun theory constants, where everything = 1
G = 1
M = 1
R = 1


# Time points for integration
t_end = 10
num_t = 300
num_points = 100
t = np.linspace(0, t_end, num_t)

# Function to compute derivatives
def derivatives(state, t):
    x, y, z, vx, vy, vz = state
    r = np.sqrt(x**2 + y**2 + z**2)

    # Acceleration of test particle
    ax = -G*M*x/r**3
    ay = -G*M*y/r**3
    az = -G*M*z/r**3

    return [vx, vy, vz, ax, ay, az]

# Initial conditions
theta0 = np.deg2rad(45.)
x0 = R*np.cos(theta0)  # Initial x position
y0 = 0.0               # Initial y position
z0 = -R*np.sin(theta0) # Initial z position
vx0 = 0.0    # Initial velocity in x direction
vy0 = np.sqrt(G*M/R)  # Initial velocity for circular orbit
vz0 = 0.0
initial_state = [x0, y0, z0, vx0, vy0, vz0]

# Integrate the equations of motion
trajectory = odeint(derivatives, initial_state, t)

# Extract positions
x = trajectory[:, 0]
y = trajectory[:, 1]
z = trajectory[:, 2]

# Create a 3D trajectory plot with color changing with x
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(color=t, colorscale='Blues', width=20),
    name='Trajectory'
))

# Add a colorbar to display the legend for time values
fig.update_traces(
    marker=dict(
        colorbar=dict(
            title="Time",
            titleside="right"
        )
    )
)

# Update layout for better visualization
fig.update_layout(
    title="3D Particle Trajectory with Time",
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis',
        xaxis = dict(nticks=4, range=[-1,1],),
        yaxis = dict(nticks=4, range=[-1, 1],),
        zaxis = dict(nticks=4, range=[-1, 1],),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)


# Generate points for the reference circle in the XY plane
theta = np.linspace(0, 2 * np.pi, num_points)
x_ref = R * np.cos(theta)
y_ref = R * np.sin(theta)
z_ref = np.zeros_like(theta)  # Circle lies in the XY plane (Z=0) here

# Add reference circle with dashed lines
fig.add_trace(go.Scatter3d(
    x=x_ref, y=y_ref, z=z_ref,
    mode='lines',
    line=dict(color='black', width=5, dash='dash'),
))

# Define arrows for x, y, and z axis
arrows = {
    "x": {"start": [0, 0, 0], "end": [1, 0, 0]},
    "y": {"start": [0, 0, 0], "end": [0, 1, 0]},
    "z": {"start": [0, 0, 0], "end": [0, 0, 1]}
    }
colors = {"x": "black",
          "y": "black",
          "z": "black"
          }

# Add arrows for each axis
for axis, data in arrows.items():
    start, end = data["start"], data["end"]
    color = colors[axis]

    # Add the arrow body (line)
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=5),
    ))

    # Add the arrowhead (cone)
    fig.add_trace(go.Cone(
        x=[end[0]],
        y=[end[1]],
        z=[end[2]],
        u=[end[0] - start[0]],
        v=[end[1] - start[1]],
        w=[end[2] - start[2]],
        sizemode="absolute",
        sizeref=0.1,
        colorscale=[[0, color], [1, color]],
        showscale=False
    ))

# Show the plot
fig.show()

# Vertical Force

Now, we introduce a vertical force
\begin{equation}
𝑭_z = - A 𝐳 ,
\end{equation}
where $A$ is a constant characterizing the strength of $F_z$.  If the test particle were not rotating about the center, this force would cause the particle oscillate about the midplane.  Does this force then align the orbit of the test particle with the midplane?

Prec_norot.svg

Below, we integrate the particle's equations of motion
\begin{equation}
\frac{d^2 𝙧}{d t^2} = - \frac{G M}{r^3} 𝙧 - A 𝙯 .
\end{equation}
Interestingly, the particle's inclination stays roughly constant with time, but its orientation continuously changes.  This is called "nodal precession."

In [4]:
# Fun theory constants, where everything = 1
G = 1
M = 1
R = 1
# Strength of vertical force
A = 0.1


# Time points for integration
t_end = 30
num_t = 300
num_points = 100
t = np.linspace(0, t_end, num_t)

# Function to compute derivatives
def derivatives(state, t):
    x, y, z, vx, vy, vz = state
    r = np.sqrt(x**2 + y**2 + z**2)
    Fz = -A*z

    # Acceleration of test particle, az includes vertical force
    ax = -G*M*x/r**3
    ay = -G*M*y/r**3
    az = -G*M*z/r**3 + Fz

    return [vx, vy, vz, ax, ay, az]

# Initial conditions
theta0 = np.deg2rad(45.)
x0 = R*np.cos(theta0)  # Initial x position
y0 = 0.0               # Initial y position
z0 = -R*np.sin(theta0) # Initial z position
vx0 = 0.0
vy0 = np.sqrt(G*M/R)   # Initial velocity for circular orbit
vz0 = 0.0
initial_state = [x0, y0, z0, vx0, vy0, vz0]

# Integrate the equations of motion
trajectory = odeint(derivatives, initial_state, t)

# Extract positions
x = trajectory[:, 0]
y = trajectory[:, 1]
z = trajectory[:, 2]

# Create a 3D trajectory plot with color changing with x
fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='lines',
    line=dict(color=t, colorscale='Blues', width=20),
    name='Trajectory'
))

# Add a colorbar to display the legend for time values
fig.update_traces(
    marker=dict(
        colorbar=dict(
            title="Time",
            titleside="right"
        )
    )
)

# Update layout for better visualization
fig.update_layout(
    title="3D Particle Trajectory with Time",
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis',
        xaxis = dict(nticks=4, range=[-1,1],),
        yaxis = dict(nticks=4, range=[-1, 1],),
        zaxis = dict(nticks=4, range=[-1, 1],),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)


# Generate points for the reference circle in the XY plane
theta = np.linspace(0, 2 * np.pi, num_points)
x_ref = R * np.cos(theta)
y_ref = R * np.sin(theta)
z_ref = np.zeros_like(theta)  # Circle lies in the XY plane (Z=0) here

# Add reference circle with dashed lines
fig.add_trace(go.Scatter3d(
    x=x_ref, y=y_ref, z=z_ref,
    mode='lines',
    line=dict(color='black', width=5, dash='dash'),
))

# Define arrows for x, y, and z axis
arrows = {
    "x": {"start": [0, 0, 0], "end": [1, 0, 0]},
    "y": {"start": [0, 0, 0], "end": [0, 1, 0]},
    "z": {"start": [0, 0, 0], "end": [0, 0, 1]}
    }
colors = {"x": "black",
          "y": "black",
          "z": "black"
          }

# Add arrows for each axis
for axis, data in arrows.items():
    start, end = data["start"], data["end"]
    color = colors[axis]

    # Add the arrow body (line)
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=5),
    ))

    # Add the arrowhead (cone)
    fig.add_trace(go.Cone(
        x=[end[0]],
        y=[end[1]],
        z=[end[2]],
        u=[end[0] - start[0]],
        v=[end[1] - start[1]],
        w=[end[2] - start[2]],
        sizemode="absolute",
        sizeref=0.1,
        colorscale=[[0, color], [1, color]],
        showscale=False
    ))

# Show the plot
fig.show()

# Physics of Precession

Why does the inclination of the particle stay constant, but its orbit normal continually change direction?  Well, lets let the vertical force decrease the particle's inclination a bit, and think about the consequences.  We assume the rotational axis of the particle is in the $x$-$z$ plane, and look down the +$y$-axis to view the particle's velocity.  We let $v_0$ describe the velocity of the particle near its maximum height, and over a small time $\Delta t$, decrease the inclination of the trajectory.

Because the particle is orbiting its central mass, its new velocity $v'$ will be the same magnitude, but point in a different direction.  The change in velocity from $v_0$ to $v'$ effectively acts to "push" the trajectory of the particle down.

Prec_Jan25_miny.svg

If we look at the particle in the opposite direction (down the $-y$-axis), we find a similar effect.  Assuming $v_0$ is the velocity of the particle at its minimum height, after the inclination decreases slightly, the new velocity $v'$ point in a different direction than $v_0$, effectively pushing the particle's trajectory down.

Prec_Jan25_posy.svg

Examining a 3D visualization of the new trajectory, rotating everywhere with velocity $v'$, after changing from a higher inclination state $v_0$, we see the sudden change pushes the circular trajectory of the particle in opposite directions when it intersects the $x$-$y$ plane, causing its orientation to change.

In [8]:
# Parameters for the circle
radius = 1
num_points = 100

# Generate points for the reference circle in the XY plane
theta = np.linspace(0, 2 * np.pi, num_points)
x_ref = radius * np.cos(theta)
y_ref = radius * np.sin(theta)
z_ref = np.zeros_like(theta)  # Circle lies in the XY plane (Z=0) here

# Tilting the circle out of the XY plane
tilt_angle = np.deg2rad(45)
x_rot = x_ref * np.cos(tilt_angle) + z_ref * np.sin(tilt_angle)
y_rot = y_ref
z_rot = -x_ref * np.sin(tilt_angle) + z_ref * np.cos(tilt_angle)

# Letting circle fall slightly
fall_angle = np.deg2rad(25)
x_fall = x_ref * np.cos(fall_angle) + z_ref * np.sin(fall_angle)
y_fall = y_ref
z_fall = -x_ref * np.sin(fall_angle) + z_ref * np.cos(fall_angle)
# Create the figure
fig = go.Figure()

# Define arrows for x, y, and z axis, and the velocity
vel = 0.7
u0_x = -vel*np.cos(tilt_angle)
u0_z = vel*np.sin(tilt_angle)
v0_x = vel*np.cos(tilt_angle)
v0_z = -vel*np.sin(tilt_angle)
u1_x = -vel*np.cos(fall_angle)
u1_z = vel*np.sin(fall_angle)
v1_x = vel*np.cos(fall_angle)
v1_z = -vel*np.sin(fall_angle)
arrows = {
    "x": {"start": [0, 0, 0], "end": [1, 0, 0]},
    "y": {"start": [0, 0, 0], "end": [0, 1, 0]},
    "z": {"start": [0, 0, 0], "end": [0, 0, 1]},
    "u0": {"start": [0, 1, 0], "end": [u0_x, 1, u0_z]},
    "v0": {"start": [0, -1, 0], "end": [v0_x, -1, v0_z]},
    "u1": {"start": [0, 1, 0], "end": [u1_x, 1, u1_z]},
    "v1": {"start": [0, -1, 0], "end": [v1_x, -1, v1_z]}
}
colors = {"x": "black",
          "y": "black",
          "z": "black",
          "u0": "blue",
          "v0": "blue",
          "u1": "purple",
          "v1": "purple"}

# Update layout for better visualization
fig.update_layout(
    title="Change in Velocity for Particle's Orbit",
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis',
        xaxis = dict(nticks=4, range=[-1,1],),
        yaxis = dict(nticks=4, range=[-1, 1],),
        zaxis = dict(nticks=4, range=[-1, 1],),
        aspectmode='cube'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Add arrows for each axis
for axis, data in arrows.items():
    start, end = data["start"], data["end"]
    color = colors[axis]

    # Add the arrow body (line)
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=5),
    ))

    # Add the arrowhead (cone)
    fig.add_trace(go.Cone(
        x=[end[0]],
        y=[end[1]],
        z=[end[2]],
        u=[end[0] - start[0]],
        v=[end[1] - start[1]],
        w=[end[2] - start[2]],
        sizemode="absolute",
        sizeref=0.1,
        colorscale=[[0, color], [1, color]],
        showscale=False
    ))

# Add fallen circle with dashed lines
fig.add_trace(go.Scatter3d(
    x=x_fall, y=y_fall, z=z_fall,
    mode='lines',
    line=dict(color='purple', width=5),
))

# Update layout for better visualization
fig.update_layout(
    scene=dict(
        aspectratio=dict(x=1, y=1, z=1),
    ),
)

# Show the plot
fig.show()

When a protoplanetary disk composed of test particles feels the force from an inclined stellar companion, the disk precesses.